In [ ]:
# 1. Install/Upgrade Necessary Libraries
# This cell might take a minute or two to run the first time.
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate safetensors torch --upgrade
!pip install invisible_watermark --upgrade # Often a dependency for some pipelines

print("Libraries installed/updated.")

# 2. Import Libraries
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import time
from diffusers import AutoencoderKL # Import the VAE loader
from google.colab import drive
print("Libraries imported.")

drive.mount('/content/drive')

In [ ]:
# 3. Check for GPU and Set Device
if not torch.cuda.is_available():
    print("WARNING: GPU not available. This model is very demanding and will be extremely slow or fail on CPU.")
    print("Please go to Runtime > Change runtime type > Hardware accelerator and select T4 GPU.")
    device = "cpu" # Fallback, but not recommended
else:
    device = "cuda"
    print(f"GPU is available. Using device: {device}")

# 4. Load the RealVisXL v5.0 Model
# This is the official model ID from Hugging Face for RealVisXL V5.0
model_id = "SG161222/RealVisXL_V5.0"
vae_model_id = "madebyollin/sdxl-vae-fp16-fix" # A commonly used high-quality SDXL VAE

print(f"\nLoading VAE: {vae_model_id}")
vae = AutoencoderKL.from_pretrained(vae_model_id, torch_dtype=torch.float16)
print("VAE loaded.")

print(f"\nLoading model: {model_id}")
print("This step will download the model (several GB) and may take 5-15 minutes the first time.")
print("Subsequent runs in the same session will be much faster.")

try:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        vae=vae, # <<< ADD THE VAE HERE
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    )
    pipe = pipe.to(device)
    print("Model loaded successfully and moved to device.")

    # Optional: Memory-saving techniques (can be useful if you run into OOM errors)
    # pipe.enable_model_cpu_offload() # Offloads parts of the model to CPU RAM when not in use
    # print("Model CPU offload enabled.")
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("xFormers memory efficient attention enabled.")
    except Exception as e:
        print(f"Could not enable xFormers: {e}. This is optional and often not critical.")

except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure your Colab runtime has enough RAM and you have a GPU selected.")
    pipe = None # Ensure pipe is None if loading failed


def load_lora_from_hf(pipe, hf_model_id, adapter_name, weight_filename=None):
    """
    Loads a LoRA from Hugging Face Hub into the provided pipeline.

    Args:
        pipe: The diffusers pipeline object.
        hf_model_id (str): The Hugging Face model ID for the LoRA (e.g., "ostris/super-cereal-sdxl-lora").
        adapter_name (str): A unique name to assign to this LoRA adapter.
        weight_filename (str, optional): The specific weight file name (e.g., "pytorch_lora_weights.safetensors")
                                         if the LoRA repo doesn't default to it or has multiple. Defaults to None.
    Returns:
        bool: True if loading was successful, False otherwise.
    """
    print(f"Attempting to load LoRA '{hf_model_id}' from Hugging Face as '{adapter_name}'...")
    try:
        if weight_filename:
            pipe.load_lora_weights(hf_model_id, weight_name=weight_filename, adapter_name=adapter_name)
        else:
            # Many LoRA repos are set up so you don't need to specify the weight_name
            pipe.load_lora_weights(hf_model_id, adapter_name=adapter_name)
        print(f"SUCCESS: LoRA '{hf_model_id}' loaded as '{adapter_name}'.")
        return True
    except Exception as e:
        print(f"ERROR loading LoRA '{hf_model_id}' from Hugging Face: {e}")
        return False

def load_lora_from_drive(pipe, lora_file_path_on_drive, adapter_name):
    """
    Loads a LoRA from a local file path (assumed to be on mounted Google Drive)
    into the provided pipeline.

    Args:
        pipe: The diffusers pipeline object.
        lora_file_path_on_drive (str): The full path to the LoRA file
           (e.g., "/content/drive/MyDrive/LoRAs/my_lora.safetensors").
        adapter_name (str): A unique name to assign to this LoRA adapter.
    Returns:
        bool: True if loading was successful, False otherwise.
    """
    print(f"Attempting to load LoRA from Drive: '{lora_file_path_on_drive}' as '{adapter_name}'...")
    if not os.path.exists(lora_file_path_on_drive):
        print(f"ERROR: File not found at '{lora_file_path_on_drive}'. Make sure Drive is mounted and path is correct.")
        return False
    try:
        # For local .safetensors, diffusers usually infers weight_name correctly
        pipe.load_lora_weights(lora_file_path_on_drive, adapter_name=adapter_name)
        print(f"SUCCESS: LoRA from '{lora_file_path_on_drive}' loaded as '{adapter_name}'.")
        return True
    except Exception as e:
        print(f"ERROR loading LoRA from '{lora_file_path_on_drive}': {e}")
        return False

# --- LoRA Loading: Multiple LoRAs ---

# # LoRA 1
# load_lora_from_hf(pipe, "user/sample_lora", "lora_name", "sample_lora.safetensors")
# lora1_trigger = "trigger1, trigger 2, "

# # LoRA 2
# load_lora_from_drive(pipe, "/content/drive/MyDrive/AI/LoRAs/SDXL/sample_lora.safetensors", "lora_name"):
# lora2_trigger = "trigger1, trigger 2, "

# --- End LoRA Loading ---

In [ ]:
# 5. Define Prompt and Parameters (Only if model loaded successfully)
if pipe:
    trigger_word = "" #lora1_trigger + lora2_trigger
    prompt = f"{trigger_word} DSLR photography of a black kitten playing with a ball of yarn." #@param {type:"string"}
    negative_prompt = "bad quality, worse quality, blurry" #@param {type:"string"}

    # SDXL is trained at 1024x1024. Other aspect ratios are fine.
    width = 1024 #@param {type:"slider", min:512, max:2048, step:64}
    height = 1024 #@param {type:"slider", min:512, max:2048, step:64}
    num_inference_steps = 30  #@param {type:"slider", min:10, max:100, step:1} # 25-40 is a good range for SDXL
    guidance_scale = 7.0 #@param {type:"slider", min:1, max:20, step:0.5} # How strongly to adhere to the prompt. 7-8.5 is common.

    print(f"\n--- Configuration ---")
    print(f"Prompt: {prompt}")
    print(f"Negative Prompt: {negative_prompt}")
    print(f"Resolution: {width}x{height}")
    print(f"Inference Steps: {num_inference_steps}")
    print(f"Guidance Scale (CFG): {guidance_scale}")
    print(f"---------------------\n")

    # 6. Generate the Image
    print("Generating image... This may take 30 seconds to a few minutes on a T4 GPU.")
    start_time = time.time()

    # Generate the image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale
    ).images[0]

    end_time = time.time()
    print(f"\nImage generated in {(end_time - start_time):.2f} seconds.")

    # 7. Display the Image
    print("Displaying image...")
    display(image) # display() is a Colab-specific function for rich output

    # To save the image (optional):
    image_filename = "realvisxl_v5_output.png"
    image.save(image_filename)
    print(f"Image saved as {image_filename}")
else:
    print("\nSkipping image generation as the model failed to load.")

In [ ]:
!nvidia-smi